In [1]:
# transfer_learning_mask_detector.py

import os
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

# === Dataset directories ===
base_data_dir = 'faces'
train_dir = os.path.join(base_data_dir, 'train')
val_dir = os.path.join(base_data_dir, 'val')
test_dir = os.path.join(base_data_dir, 'test')

# === Parameters ===
img_size = (128, 128)
batch_size = 32
num_classes = len(os.listdir(train_dir))

# === Data generators ===
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = val_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

# === Load base model ===
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# === Train model ===
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[early_stop]
)

model.save('mask_model_mobilenetv2.h5')
print("\n✅ Model trained and saved as mask_model_mobilenetv2.h5")

# === Predict and label test images ===
output_dir = "labeled_test_images"
os.makedirs(output_dir, exist_ok=True)

class_names = list(train_data.class_indices.keys())
model = load_model("mask_model_mobilenetv2.h5")

for i in range(len(test_data.filenames)):
    img_path = os.path.join(test_dir, test_data.filenames[i])
    original_image = cv2.imread(img_path)
    if original_image is None:
        continue

    image = cv2.resize(original_image, img_size)
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)

    pred = model.predict(image)[0]
    class_id = np.argmax(pred)
    label = class_names[class_id]
    confidence = pred[class_id]

    label_text = f"{label} ({confidence:.2f})"
    (text_w, text_h), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
    cv2.rectangle(original_image, (5, 5), (5 + text_w + 10, 5 + text_h + 20), (0, 0, 0), -1)
    cv2.putText(original_image, label_text, (10, 10 + text_h), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    save_path = os.path.join(output_dir, os.path.basename(img_path))
    cv2.imwrite(save_path, original_image)
    print(f"✅ Saved labeled image: {save_path}")

print("🎉 Done labeling test images!")


Found 293 images belonging to 3 classes.
Found 97 images belonging to 3 classes.
Found 39 images belonging to 3 classes.


c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.6502 - loss: 1.1851

c:\Users\ASUS\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 486ms/step - accuracy: 0.6587 - loss: 1.1592 - val_accuracy: 0.8557 - val_loss: 0.5138
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 286ms/step - accuracy: 0.8634 - loss: 0.5229 - val_accuracy: 0.9175 - val_loss: 0.3215
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 275ms/step - accuracy: 0.9013 - loss: 0.4564 - val_accuracy: 0.9588 - val_loss: 0.2470
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 275ms/step - accuracy: 0.9118 - loss: 0.4108 - val_accuracy: 0.9588 - val_loss: 0.2058
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 271ms/step - accuracy: 0.9086 - loss: 0.3167 - val_accuracy: 0.9588 - val_loss: 0.1833
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 282ms/step - accuracy: 0.9509 - loss: 0.1750 - val_accuracy: 0.9588 - val_loss: 0.1798
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 297ms/step - accuracy: 0.9601 - loss: 0.1838 - val_accuracy: 0.9588 - val_loss: 0.1764
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 281ms/step - accuracy: 0.9548 - loss: 0.1832 - val_accuracy: 0.9691 - val


✅ Model trained and saved as mask_model_mobilenetv2.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 926ms/step
✅ Saved labeled image: labeled_test_images\mask_weared_incorrect_5.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
✅ Saved labeled image: labeled_test_images\mask_weared_incorrect_6.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ Saved labeled image: labeled_test_images\with_mask_103.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ Saved labeled image: labeled_test_images\with_mask_120.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
✅ Saved labeled image: labeled_test_images\with_mask_136.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
✅ Saved labeled image: labeled_test_images\with_mask_14.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
✅ Saved labeled image: labeled_test_images\with_mask_149.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
✅ Saved labeled image: labeled_test_images\with_mask_15.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
✅ Saved labeled image: labeled_test_images\with_mask_151.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
✅ Saved labeled image: labeled_test_images\with_mask

In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the trained classification model
model = load_model("mask_model_mobilenetv2.h5")  # or "mask_model.keras"

# Class names in training order
class_names = ['mask_weared_incorrect', 'with_mask', 'without_mask']

# Load OpenCV face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Input/output directories
input_folder = "test_images"
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

# Process each image
for filename in os.listdir(input_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    img_path = os.path.join(input_folder, filename)
    image = cv2.imread(img_path)
    if image is None:
        print(f"❌ Could not read {filename}")
        continue

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    print(f"🔍 Found {len(faces)} face(s) in {filename}")

    for (x, y, w, h) in faces:
        face = image[y:y + h, x:x + w]
        if face.size == 0:
            continue

        face_resized = cv2.resize(face, (128, 128))
        face_normalized = face_resized.astype("float32") / 255.0
        face_input = img_to_array(face_normalized)
        face_input = np.expand_dims(face_input, axis=0)

        pred = model.predict(face_input)[0]
        class_id = np.argmax(pred)
        label = class_names[class_id]
        confidence = pred[class_id]

        color = (0, 255, 0) if label == "with_mask" else (0, 0, 255)
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

        label_text = f"{label} ({confidence:.2f})"
        (text_w, text_h), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        label_y = y - 10 if y - 10 > text_h else y + h + 20
        cv2.rectangle(image, (x, label_y - text_h - 4), (x + text_w, label_y + 4), color, -1)
        cv2.putText(image, label_text, (x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    save_path = os.path.join(output_folder, filename)
    cv2.imwrite(save_path, image)
    print(f"✅ Saved result: {save_path}")

print("🎉 Done. All labeled images saved in 'output/' folder.")


🔍 Found 2 face(s) in images_2.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
✅ Saved result: output\images_2.jpeg
🔍 Found 3 face(s) in images_4.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
✅ Saved result: output\images_4.jpeg
🔍 Found 2 face(s) in image_0.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
✅ Saved result: output\image_0.jpeg
🔍 Found 1 face(s) in image_1.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
✅ Saved result: output\image_1.jpeg
🔍 Found 0 face(s) in image_3.jpeg
✅ Saved result: output\image_3.jpeg
🔍 Found 1 face(s) in image_5.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
✅ Saved result: output\image_5.jpeg
🔍 Found 0 face(s) in image_6.jpeg
✅ Saved result: output\image_6.jpeg
🔍 Found 0 face(s) in image_7.jpeg
✅ Saved result: output\image_7.jpeg
🔍 Found 0 face(s) in mask_weared_incorrect_5.jpg
✅ Saved result: output\mask_weared_incorrect_5.